## Day 12: Feature Matching and the Ratio Test

#### Goals
* Know the major feature detector/descriptor algorithms available in OpenCV.
* Understand how to match features between images using the SSD metric.
* Understand the ratio test and why it's critical for finding good matches.
* Be able to implement feature matching and apply the ratio test.

In [ ]:
# boilerplate setup
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import cv2

### Recap: The Feature Pipeline

Over the past few classes, we've been building up the pieces for feature-based computer vision:

1. **Detection**: Find interesting keypoint locations (Harris corners)
2. **Description**: Extract a feature vector at each keypoint (MOPS descriptor)
3. **Matching**: Compare descriptors between images to find correspondences ← **Today's focus!**

With matching, we'll finally be able to find the **same physical points** across different images - the foundation for panorama stitching, 3D reconstruction, tracking, and more.

## OpenCV Feature Detectors

We already learned about the Harris Corners feature detector and the MOPS feature descriptor. These are "student-level" tech--a good introduction to the ideas, but not the most performant tech available. Before we start matching we'll survey the various feature detectors and descriptors you might want to use in OpenCV. 

Note that some of these detectors and descriptors come as joined pairs, and others can be mixed and matched.
They have different tradeoffs between speed, accuracy, patent restrictions, and invariance properties.

### Harris Corners (OpenCV Built-in)

`cv2.cornerHarris()`

**What it is:** The classic corner detector we've been implementing from scratch, available in OpenCV.

**Key characteristics:**
* Detects corners by analyzing eigenvalues of the "structure tensor"
* Returns corner **locations only** (not a combined detector/descriptor like SIFT)
* No scale invariance built-in
* Can be combined with any descriptor (MOPS, SIFT descriptors, etc.)

**Advantages:**
* Fast corner detection
* Well-understood classical algorithm
* Good for tracking and simple scenarios
* Can mix with any descriptor

**Disadvantages:**
* Only detects - doesn't describe (need separate descriptor)
* Not scale invariant (corners "repeat" at different pyramid levels)
* Less robust than modern integrated detector/descriptors

**When to use:** Learning, simple tracking applications, or when you want to experiment with separating detection and description

In [ ]:
# Harris corner detection example
img = cv2.imread("../data/yos1.jpg", cv2.IMREAD_GRAYSCALE)

# cv2.cornerHarris - returns corner response map
harris_response = cv2.cornerHarris(img, blockSize=5, ksize=3, k=0.04)

# Threshold to find corner peaks
threshold = 0.05 * harris_response.max()
corner_mask = harris_response > threshold

# Find corner coordinates
corner_coords = np.argwhere(corner_mask)
print(f"cv2.cornerHarris found {len(corner_coords)} corners above threshold")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Show harris response map
axes[0].imshow(harris_response, cmap='viridis')
axes[0].set_title("Harris Corner Response Map", fontsize=12)
axes[0].axis('off')

# Show cornerHarris detections
img_harris = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
for coord in corner_coords[::10]:  # Show every 10th corner to avoid clutter
    cv2.circle(img_harris, (coord[1], coord[0]), 3, (255, 0, 0), -1)
axes[1].imshow(img_harris)
axes[1].set_title(f"Detected Corners\n(showing subset of {len(corner_coords)} total)", fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

### SIFT (Scale-Invariant Feature Transform)

**What it is:** The "gold standard" feature detector/descriptor developed by David Lowe in 1999.

**Key characteristics:**
* Uses Difference of Gaussians (DoG) for keypoint detection across scale space
* Produces 128-dimensional descriptors
* Highly robust to scale, rotation, and lighting changes
* Selects a **characteristic scale** for each feature (unlike Harris, where each corner tends to be detected at many scales)

**Advantages:**
* Most distinctive and reliable feature matching
* Excellent scale and rotation invariance
* Patent expired in 2020 - now freely available!

**Disadvantages:**
* Not the fastest option
* Higher memory usage (128-D floats)

**When to use:** High-quality matching where accuracy matters more than speed (e.g., Structure from Motion, precise object recognition)

In [ ]:
# SIFT example
img = cv2.imread("../data/yos1.jpg", cv2.IMREAD_GRAYSCALE)

# Create SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(img, None)

print(f"SIFT detected {len(keypoints)} keypoints")
print(f"Descriptor shape: {descriptors.shape} (N × 128 float32)")
print(f"First keypoint: location=({keypoints[0].pt[0]:.1f}, {keypoints[0].pt[1]:.1f}), size={keypoints[0].size:.1f}, angle={keypoints[0].angle:.1f}°")

# Visualize keypoints (size = scale, angle = orientation)
img_with_kp = cv2.drawKeypoints(img, keypoints, None, 
                                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_with_kp, cv2.COLOR_BGR2RGB))
plt.title(f"SIFT Features", fontsize=14)
plt.axis('off')
plt.show()

### ORB (Oriented FAST and Rotated BRIEF)

**What it is:** A fast, free alternative to SIFT, developed by OpenCV Labs in 2011.

**Key characteristics:**
* Combines FAST corner detector with BRIEF descriptor
* Produces 256-bit binary descriptors (32 bytes)
* Adds rotation invariance to BRIEF via orientation computation
* Much faster than SIFT

**Advantages:**
* **Fast** - suitable for real-time applications
* **Free** - no patent restrictions
* Binary descriptors enable very fast matching (Hamming distance)
* Low memory footprint

**Disadvantages:**
* Less accurate than SIFT
* Not as scale-invariant as SIFT

**When to use:** Real-time applications, mobile/embedded devices, or when you need free/open-source

In [ ]:
# ORB example
orb = cv2.ORB_create()
keypoints_orb, descriptors_orb = orb.detectAndCompute(img, None)

print(f"ORB detected {len(keypoints_orb)} keypoints")
print(f"Descriptor shape: {descriptors_orb.shape} (N × 32 uint8, representing 256 bits)")
print(f"First descriptor (binary): {descriptors_orb[0]}")

img_with_orb = cv2.drawKeypoints(img, keypoints_orb, None, color=(0, 255, 0),
                                 flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_with_orb, cv2.COLOR_BGR2RGB))
plt.title(f"ORB Features ({len(keypoints_orb)} keypoints)", fontsize=14)
plt.axis('off')
plt.show()

### BRISK (Binary Robust Invariant Scalable Keypoints)

**What it is:** A binary feature detector/descriptor designed for efficiency (2011).

**Key characteristics:**
* Uses AGAST (faster variant of FAST) for detection
* Binary descriptor based on sampling patterns
* Scale-invariant via scale-space pyramid

**Advantages:**
* Very fast computation
* Good rotation and scale invariance
* Binary = fast matching with Hamming distance
* Patent-free

**Disadvantages:**
* Less distinctive than SIFT
* Similar performance to ORB in many cases

**When to use:** Another good option for real-time applications, alternative to ORB

In [ ]:
# BRISK example (limited to 500 keypoints)
brisk = cv2.BRISK_create()
keypoints_brisk_all, descriptors_brisk_all = brisk.detectAndCompute(img, None)

# Take only the top 500 keypoints by response strength
keypoints_brisk_sorted = sorted(keypoints_brisk_all, key=lambda x: x.response, reverse=True)
keypoints_brisk = keypoints_brisk_sorted[:500]
descriptors_brisk = descriptors_brisk_all[:500]

print(f"BRISK detected {len(keypoints_brisk_all)} keypoints total")
print(f"Showing top {len(keypoints_brisk)} by response strength")
print(f"Descriptor shape: {descriptors_brisk.shape} (N × 64 uint8 binary)")

img_with_brisk = cv2.drawKeypoints(img, keypoints_brisk, None, color=(0, 165, 255),
                                   flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_with_brisk, cv2.COLOR_BGR2RGB))
plt.title(f"BRISK Features (showing top {len(keypoints_brisk)} of {len(keypoints_brisk_all)} keypoints)", fontsize=14)
plt.axis('off')
plt.show()

### Quick Comparison

| Algorithm | Descriptor Type | Descriptor Size | Speed | Distinctiveness | Patent-Free | Best For |
|-----------|----------------|-----------------|-------|-----------------|-------------|----------|
| **SIFT** | Float | 128-D (512 bytes) | Slow | ★★★★★ | ✓ (since 2020) | High-quality matching, SfM |
| **ORB** | Binary | 256 bits (32 bytes) | Fast | ★★★ | ✓ | Real-time, mobile apps |
| **AKAZE** | Binary | 488 bits (61 bytes) | Medium | ★★★★ | ✓ | Good balance of speed/quality |
| **BRISK** | Binary | 512 bits (64 bytes) | Fast | ★★★ | ✓ | Real-time alternative to ORB |

**For this class:** We'll use SIFT for the rest of today's lesson since it produces the most reliable matches. In your projects, feel free to experiment with different algorithms!

---

## Feature Matching

Now we have two images, each with a set of feature descriptors. **How do we find which features correspond to the same physical point?**

The basic idea: features that describe the same point should have **similar descriptors**.

So we need:
1. A way to measure how different two descriptors are
2. An algorithm to find corresponding pairs

### SSD: Sum of Squared Differences

The simplest distance metric for comparing two feature vectors $\mathbf{f}$ and $\mathbf{g}$ is the **Sum of Squared Differences (SSD)**:

$$
\text{SSD}(\mathbf{f}, \mathbf{g}) = \sum_{i=1}^d (f_i - g_i)^2
$$

where $d$ is the dimension of the descriptor.

* **Small SSD** = descriptors are similar = likely a match
* **Large SSD** = descriptors are different = not a match

This is equivalent to squared Euclidean distance. We've seen this before in the Harris corner metric!

In [ ]:
def ssd(f, g):
    """Compute sum of squared differences between two feature vectors."""
    return np.sum(np.square(f - g))

### Toy Example: Matching Two Sets of Features

Let's work through a small example by hand (and with code). Suppose we have:
* Image 1 with 4 features: `F1[:, i]` for i=0,1,2,3
* Image 2 with 3 features: `F2[:, j]` for j=0,1,2

Each feature is a 2-dimensional vector (normally they'd be 64-D or 128-D, but we'll keep it simple).

In [ ]:
# Two sets of toy features (each column is one feature)
F1 = np.array([
    [0, 1, 4, 3],
    [1, 0, 4, 1]], dtype=np.float32)

F2 = np.array([
    [2, 5, 1],
    [1, 5, 2]], dtype=np.float32)

print("F1 shape:", F1.shape, "(2 dimensions × 4 features)")
print("F2 shape:", F2.shape, "(2 dimensions × 3 features)")
print("\nF1 features as columns:")
print(F1)
print("\nF2 features as columns:")
print(F2)

##### Discussion Question

How would you find which feature in F2 is closest to each feature in F1?

What data structure would help organize this?

### Building a Distance Matrix

The most straightforward approach: compute **all pairwise distances** in a matrix.

Create a 4×3 matrix where entry $(i,j)$ contains `SSD(F1[:, i], F2[:, j])`.

Then finding the closest match for each feature in F1 is just finding the minimum in each row!

In [ ]:
# Compute all pairwise distances
n1 = F1.shape[1]  # number of features in F1
n2 = F2.shape[1]  # number of features in F2

distances = np.zeros((n1, n2))
for i in range(n1):
    for j in range(n2):
        distances[i, j] = ssd(F1[:, i], F2[:, j])

print("Distance matrix (F1 features × F2 features):")
print(distances)
print("\nRows = features in F1, Columns = features in F2")
print("Entry (i,j) = SSD between F1[:,i] and F2[:,j]")

In [ ]:
# For each feature in F1, find the closest feature in F2
closest_matches = np.argmin(distances, axis=1)
match_distances = np.min(distances, axis=1)

print("Matches (F1 → F2):")
for i in range(n1):
    print(f"  F1[{i}] → F2[{closest_matches[i]}]  (SSD = {match_distances[i]:.1f})")

### Computational Complexity

**Brute force matching:** Compare every feature in image 1 to every feature in image 2.

* If each image has $n$ features with $d$-dimensional descriptors
* We compute $n \times n$ distances
* Each distance computation costs $O(d)$
* **Total:** $O(n^2 d)$

For real images: $n \sim 1000$ features, $d \sim 128$ dimensions → millions of operations!

**Optimizations:**
* Vectorized distance computation: `scipy.spatial.distance.cdist` (or for binary: `cv2.BFMatcher` with Hamming distance)
* Approximate nearest neighbors: kd-trees, FLANN, etc.
* For today: we'll stick with brute force for clarity

### The Problem with Closest Match

**Naive algorithm:** For each feature in image 1, take its closest match in image 2.

**Problem:** What if the closest match isn't very close? 

Consider a repetitive texture like a fence:

In [ ]:
# Load the fence image
fence = cv2.imread("../data/fences.jpg")
fence_rgb = cv2.cvtColor(fence, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(14, 6))
plt.imshow(fence_rgb)
plt.title("The Fence Strikes Back: Why we can't just take the closest match", fontsize=14)
plt.axis('off')
plt.show()

In this image, many fence posts look nearly identical. A feature on one post will have:
* Its **true match** on the corresponding post in the other image
* Many **similar-looking features** on other posts

If we just take the closest match, we might pick the wrong post! Even worse: the closest match distance won't tell us if we should trust the match.

**Key insight:** Good matches should have a closest match that is **much closer** than the second-closest match.

## The Ratio Test

**Lowe's Ratio Test** (from the SIFT paper): Don't just look at the closest match - look at the **ratio** between the closest and second-closest matches.

For a feature $\mathbf{f}$ in image 1, let:
* $\mathbf{g}_1$ = closest match in image 2 (smallest SSD)
* $\mathbf{g}_2$ = second-closest match in image 2 (second smallest SSD)

Define the **ratio distance**:

$$
d_{\text{ratio}} = \frac{\text{SSD}(\mathbf{f}, \mathbf{g}_1)}{\text{SSD}(\mathbf{f}, \mathbf{g}_2)}
$$

**Accept the match** only if $d_{\text{ratio}} < \tau$ (typically $\tau \approx 0.8$).

##### Discussion Questions

1. What does $d_{\text{ratio}} = 1.0$ mean?

2. What does $d_{\text{ratio}} = 0.5$ mean?

3. Why is a **small** ratio distance a sign of a good match?

4. What would happen on a feature from a repetitive fence post?

### Ratio Test Intuition

**Good matches** (distinctive features):
* Closest match is much closer than second-closest
* Ratio distance is **small** (e.g., 0.5)
* We're confident this is the right match!

**Bad matches** (ambiguous/repetitive features):
* Closest and second-closest are similar distances
* Ratio distance is **close to 1.0**
* Could match to multiple features - reject it!

By rejecting ambiguous matches, we get fewer but **higher quality** correspondences.

### Implementing the Ratio Test

In [ ]:
# Define our toy feature sets
F1 = np.array([
    [0, 1, 4, 3],
    [1, 0, 4, 1]], dtype=np.float32)

F2 = np.array([
    [2, 5, 1],
    [1, 5, 2]], dtype=np.float32)

n1 = F1.shape[1]  # 4 features in F1
n2 = F2.shape[1]  # 3 features in F2
d  = F1.shape[0]  # feature dimensionality

# Define SSD distance function
def ssd(f, g):
    return np.sum(np.square(f - g))

# Compute all pairwise distances
distances = np.zeros((n1, n2))
for i in range(n1):
    for j in range(n2):
        distances[i, j] = ssd(F1[:, i], F2[:, j])

# Find closest matches for each feature in F1
closest_matches = np.argmin(distances, axis=1)

# Compute ratio distances
ratio_distances = np.zeros(n1)
for i in range(n1):
    # Get sorted distances for feature i in F1
    sorted_distances = np.sort(distances[i, :])
    
    # Ratio of closest to second-closest
    d1 = sorted_distances[0]  # closest
    d2 = sorted_distances[1]  # second-closest
    
    ratio_distances[i] = d1 / d2 if d2 > 0 else 0

print("Ratio test results (F1 → F2):")
for i in range(n1):
    print(f"  F1[{i}] → F2[{closest_matches[i]}]  (SSD = {distances[i, closest_matches[i]]:.1f}, ratio = {ratio_distances[i]:.3f})")

# Apply ratio test threshold
ratio_threshold = 0.3
good_matches = ratio_distances < ratio_threshold

print(f"\nMatches passing ratio test (threshold = {ratio_threshold}):")
for i in range(n1):
    if good_matches[i]:
        print(f"  F1[{i}] → F2[{closest_matches[i]}] ✓")
    else:
        print(f"  F1[{i}] → (rejected) ✗")

---

## Closing Demo: Matching Real Images

Our plan:
1. Load two overlapping images
2. Detect SIFT features in both images
3. Match features using OpenCV's built-in matcher
4. Apply the ratio test
5. Visualize the results

In [ ]:
# Load Yosemite images
yos1 = cv2.imread("../data/yos1.jpg")
yos2 = cv2.imread("../data/yos2.jpg")

yos1_gray = cv2.cvtColor(yos1, cv2.COLOR_BGR2GRAY)
yos2_gray = cv2.cvtColor(yos2, cv2.COLOR_BGR2GRAY)

# Display both images
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].imshow(cv2.cvtColor(yos1, cv2.COLOR_BGR2RGB))
axes[0].set_title("Yosemite Image 1", fontsize=14)
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(yos2, cv2.COLOR_BGR2RGB))
axes[1].set_title("Yosemite Image 2", fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Image 1 shape: {yos1.shape}")
print(f"Image 2 shape: {yos2.shape}")

In [ ]:
# Detect SIFT features in both images
sift = cv2.SIFT_create()

kp1, desc1 = sift.detectAndCompute(yos1_gray, None)
kp2, desc2 = sift.detectAndCompute(yos2_gray, None)

print(f"Image 1: detected {len(kp1)} SIFT keypoints")
print(f"Image 2: detected {len(kp2)} SIFT keypoints")
print(f"Descriptor dimensions: {desc1.shape[1]}")

In [ ]:
# Match features using brute-force matcher
# We use knnMatch with k=2 to get the top 2 matches for ratio test
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches_knn = bf.knnMatch(desc1, desc2, k=2)

print(f"Found {len(matches_knn)} features with at least 2 matches")

In [ ]:
# Apply ratio test
ratio_threshold = 0.75
good_matches = []

for match_pair in matches_knn:
    if len(match_pair) == 2:  # Make sure we have 2 matches
        m, n = match_pair  # m = closest, n = second-closest
        # m.distance is the SSD to closest match
        # n.distance is the SSD to second-closest match
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)

print(f"Total candidate matches: {len(matches_knn)}")
print(f"Matches passing ratio test: {len(good_matches)}")
print(f"Rejection rate: {100 * (1 - len(good_matches)/len(matches_knn)):.1f}%")

In [ ]:
# Visualize ALL matches (before ratio test)
all_matches_img = cv2.drawMatches(
    yos1, kp1, yos2, kp2, 
    [m[0] for m in matches_knn if len(m) == 2],  # take closest match from each pair
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

plt.figure(figsize=(20, 8))
plt.imshow(cv2.cvtColor(all_matches_img, cv2.COLOR_BGR2RGB))
plt.title(f"All Closest Matches (Before Ratio Test): {len(matches_knn)} matches", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize GOOD matches (after ratio test)
good_matches_img = cv2.drawMatches(
    yos1, kp1, yos2, kp2, good_matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

plt.figure(figsize=(20, 8))
plt.imshow(cv2.cvtColor(good_matches_img, cv2.COLOR_BGR2RGB))
plt.title(f"Good Matches (After Ratio Test, threshold={ratio_threshold}): {len(good_matches)} matches", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

Compare the two visualizations above:

**Before ratio test:** Many matches, but some are clearly wrong (crossing lines, misaligned features)

**After ratio test:** Fewer matches, but they're (mostly) correct!


#### The Ratio Test Threshold

(Try changing the `ratio_threshold` value in the cell below and re-run the matching visualization.)

* **Lower threshold (e.g., 0.5):** Fewer but higher-confidence matches
* **Higher threshold (e.g., 0.9):** More matches but lower quality

Lowe's original SIFT paper recommended 0.8. For panorama stitching, 0.7-0.75 often works well.

## Summary

Today we completed the feature matching pipeline:

1. **OpenCV Feature Detectors**: SIFT (best quality), ORB (fast/free), BRISK (fast/free)

2. **Feature Matching**: Compare descriptors using distance metrics like SSD (sum of squared differences)

3. **The Ratio Test**: Accept a match only if the closest match is significantly better than the second-closest